# The instability of numerical codes: symptoms

The instability of numerical schemes and codes is one of the critical problems that a researcher using numerical experimentation may encounter. In this exercise, we are going to see how violent numerical instabilities can be.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from importlib import reload
import import_ipynb

from ex_2b import get_uu_t0, get_u_exact
from nm_lib import nm_lib as nm, utils as utils
# Define some simple global matplotlib settings
plt.rcParams.update(
    {"font.size" : "15",
     "font.family"          : "STIXGeneral",
     "mathtext.fontset"     : "stix",
     "figure.autolayout"    : "True",
     "figure.figsize"       : (7,7),
     "lines.markersize"     : 10,
     }
)

def animate_u(uunt: np.ndarray, xx: np.ndarray, tt: np.ndarray, a:float = None, exact: callable = None, initial = None) -> None:
    """
    Animates a function u(x,t) in the interval xx over time tt.

    Arguments:
        uunt {np.ndarray} -- [len(tt),len(xx)] array with the solution of u(x,t)
        xx {np.ndarray} -- spacial coordinate
        tt {np.ndarray} -- time array
        a {float or None} -- advection speed, if not constant pass in None

    Keyword Arguments:
        exact {callable} -- if given, a callable returning the 1D array with exact solution, signature (x,t) (default: {None})
        initial {np.ndarray or None} -- if given, the initial state of uunt (default: {None})
    """
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
    title_str_app = f" a={a:.2f}" if a is not None else ""
    print(f"Animation with nint={len(xx)-1:d}"+title_str_app+f", nframes={len(tt):d}:")

    def init(): 
        ax.plot(xx,uunt[0,:])

    def animate(i):
        ax.clear()
        ax.plot(xx,uunt[i,:], label="uunt")
        if initial is None:
            ax.plot(xx,uunt[0,:], ls=":", label="init")
        else:
            ax.plot(xx, initial, ls=":", label="init")
        if exact is not None:
            ax.plot(xx, exact(xx, tt[i],a),ls="--", label="exact")
        ax.set_title(f"t={tt[i]:.2f}, nint={len(xx)-1:d}"+title_str_app)
        ax.legend(loc=1)

    anim = FuncAnimation(fig, animate, interval=200, frames=len(tt), init_func=init)
    html = HTML(anim.to_jshtml())
    display(html)
    plt.close()

def analyse_instability(nint: int, nt: int = 60, x0: float = -2.6, xf: float = 2.6, a=1, crit_value: float = 1.5, **kwargs):
    """
    A very(!) messy function for the analysis performed in this exercise. This is implemented and adjusted on the
    go dependent on what is needed in the specific case. Functions like this is not safe... Instead of a normal
    docstring I've tried to write out the most important parts:

    It basically initiates the required xx-array, then handles the input keyword arguments which is specifying
    the type of derivative, bnd_limits and the various functions to be used for evolving in time.

    After finding the solution uunt, it checks for instability. Instability at specific timestep is defined 
    if the maximum absolute value of uunt at that timestep is larger than input `crit_value`. If an instability
    of this kind is found, it prints the time step id and the actuall time. The function then animates the solution
    at every 4th timestep from the initial state until the end (or until the instability point) if desired.
    Finally the time of instability, index of instability, the time array, solution array and xx-array is returned.
    """
    xx, dx = utils.get_xx(nint, x0, xf)

    # Handle additional keyword arguments
    ddx = kwargs["ddx"] if "ddx" in kwargs else nm.deriv_dnw
    bnd_limits = kwargs["bnd_limits"] if "bnd_limits" in kwargs else [0,1]
    cfl_cut = kwargs["cfl_cut"] if "cfl_cut" in kwargs else 0.98
    ani = kwargs["animate"] if "animate" in kwargs else True
    evolv_func = kwargs["evolv_func"] if "evolv_func" in kwargs else nm.evolv_adv_burgers
    init_func = kwargs["init_func"] if "init_func" in kwargs else get_uu_t0
    u_exact = kwargs["u_exact"] if "u_exact" in kwargs else get_u_exact

    uut0 = init_func(xx)
    if a is None:
        tt, uunt = evolv_func(xx, uut0, nt, ddx=ddx, bnd_limits=bnd_limits, cfl_cut=cfl_cut)
    else:
        tt, uunt = evolv_func(xx, uut0, a, nt, ddx=ddx, bnd_limits=bnd_limits, cfl_cut=cfl_cut)
    # check for instability:
    crit_value = crit_value    # I say instability occurs if any values in u(x,t) is larger than this value
    id_crit = np.argmax(np.max(np.abs(uunt),axis=1)>crit_value) # find first idx of time where the max abs value in uunt is larger than crit_value
    print(f"Using {ddx.__name__}, with {nint:d} intervals, cfl_cut={cfl_cut:.2f}, nt={nt:d}")
    if id_crit == 0:
        print(f"Instability not found for t<{tt[-1]:.2f}")
        crit_time = None
        anim_idt = None
    else:
        crit_time = tt[id_crit]
        anim_idt = id_crit + 3
        print(f"Instability after {id_crit:d} timesteps, t={crit_time:.2f}")
    if ani is True:
        animate_u(uunt[:anim_idt:4,:], xx, tt[:anim_idt:4], a, exact=u_exact)
    elif ani == "last":
        animate_u(uunt[anim_idt-1:anim_idt,:], xx, tt[anim_idt-1:anim_idt], a, exact=u_exact, initial=uunt[0,:])
    return crit_time, id_crit, tt, uunt, xx


## 1 – Numerical instability: violent development

1. Repeat the numerical simulation carried out __in exercise [ex_2a](https://github.com/AST-Course/AST5110/blob/main/ex_2a.ipynb)__, but now take $a = 1$ (it was $a = −1$); (use a moderate number of intervals, like, e.g., 128). Check out what happens. After how many timesteps does instability become evident?

Repeat the experiment with 1024 points (and afterward with any other power of 2 you may want to use). Is the experiment stable now? After how many timesteps does the instability become evident now?

2. Still for $a = 1$, use for the spatial differentiation backward finite differencing, i.e.:

$$\left(\frac{\partial u}{\partial x}\right)_{x=x_i} \rightarrow \frac{u_i-u_{i-1}}{\Delta x}  \tag{1}$$

Fill in the `nm_lib` function `deriv_upw`, and use the `lambda` function `ddx` and select the proper limits for `bnd_limits`. 

__taking care of also changing the implementation of the boundary condition.__ Does the unstable character of the code change?

<span style="color:Orange">

## Comments

In this exercise I've gradually developed the function `analyse_instability`. The result is a quite messy function, but hopefully it's purpose is understandable. It more or less computes the solution I want given a set of input for the function and it's derivative, the numerical scheme and boundary conditions. After evolving the solutions, instead of manually inspecting the animation for instability, I perform a check of the maximum absolute value and compare it to a specified critical value also given as input. The appropriate input for this critical value is of course dependent on the problem. For the first part of the exercise I've chosen `crit_value=1.5` which corresponds to the function-value being increased by a factor of $1.5$ of it's exact value.

### Testing instability with downwind:

In [ ]:
if __name__ == "__main__":
    analyse_instability(128)
    analyse_instability(1024)
    analyse_instability(1024*2, animate=False)
    analyse_instability(1024*4, animate=False)
    analyse_instability(1024*8, animate=False)

<span style="color:Orange">

## Comments

Here we see that the solution becomes unstable after very few time steps. In the case of `nint=128` it only takes 8 time steps before the peak of the propagating function to reach the critical value of 1.5. As we know from the analytical solution, the amplitude of the curve should not change - so the increase indicates instability and that we are on the onset of the error being magnified exponentially.

Interestingly, for `nint=1028` and above, we find the exact same behavior for higher resolution. The function start to look "fuzzy" before exploding at time step 34. So this is not a type of instability we can fix by just increasing the resolution, but looks like an inherit property of applying the wrong scheme.</span>

### Testing instability with upwind:

In [ ]:
if __name__ == "__main__":
    analyse_instability(64, nt=120, ddx=nm.deriv_upw, bnd_limits=[1, 0], animate=False)
    analyse_instability(64 * 2, nt=120 * 2, ddx=nm.deriv_upw, bnd_limits=[1, 0])
    analyse_instability(64 * 4, nt=120 * 4, ddx=nm.deriv_upw, bnd_limits=[1, 0], animate=False)
    analyse_instability(64 * 8, nt=120 * 8, ddx=nm.deriv_upw, bnd_limits=[1, 0], animate=False)
    analyse_instability(64 * 16, nt=120 * 16, ddx=nm.deriv_upw, bnd_limits=[1, 0], animate=False)


<span style="color:Orange">

## Comments

Using the upwind method I find no instabilities regardless of resolution in space.

## 2 – Centered differences

Use now for the spatial derivation _centered finite differences_, i.e.:

$$\left(\frac{\partial u}{\partial x}\right)_{x=x_i} \rightarrow \frac{u_{i+1}-u_{i-1}}{2\Delta x}   \tag{2}$$

(and see the note on boundary conditions below). Fill in `nm_lib` the function `deriv_cent`, and like in the previous case, use the `lambda` function `ddx` and select the proper limits for `bnd_limits`. Use `cfl_cut=0.3`. Is any instability apparent in this case? Does the situation change when you change the sign of the constant $a$?

Note: In this case, the periodicity boundary condition can be implemented as follows: define $xx$ so that the endpoint $x = x_0$ of the domain coincides with $xx[1]$ (i.e., the second component of the array) and the endpoint $x = x_f$ coincides with the last element in the array [i.e., $xx[nump-1]$]. For the boundary condition, you can do the following: assume you are calling $uun$ the array at time $t + \Delta t$. Then the boundary condition is imposed by specifying $uun[0] = uun[nump-2]$ and $uun[nump-1] = uun[1]$.

In [ ]:
if __name__ == "__main__":
    nt = 2000
    analyse_instability(64, nt=nt,a=-1,ddx= nm.deriv_cent, bnd_limits=[1,1], cfl_cut=0.3, animate=False)
    analyse_instability(128,nt=nt,a=1, ddx= nm.deriv_cent, bnd_limits=[1,1], cfl_cut=0.3, animate=True)
    analyse_instability(128,nt=nt,a=-1,ddx= nm.deriv_cent, bnd_limits=[1,1], cfl_cut=0.3, animate=True)
    analyse_instability(256,nt=nt,a=-1,ddx= nm.deriv_cent, bnd_limits=[1,1], cfl_cut=0.3, animate=False)
    analyse_instability(512,nt=nt,a=-1,ddx= nm.deriv_cent, bnd_limits=[1,1], cfl_cut=0.3, animate=False)
    analyse_instability(1028,nt=nt,a=-1,ddx= nm.deriv_cent, bnd_limits=[1,1], cfl_cut=0.3, animate="last")

<span style="color:Orange">

## Comments
For the centered difference scheme I find no configuration of parameters resulting in a stable solution. I think this is due to the time derivative and spacial derivative being evaluated at different grid points. So for the centered differencing we have to adopt another approximation for the time derivative. The result with the present scheme looks somewhat like the unstable solutions we had previously from using the non-centered schemes in the wrong direction. The propagating function develops an oscillating tail behind it self in the direction of motion, where the solution should be equal to zero. By increasing the number of grid points to 1028, we again get this "fuzzy" solution which is magnified exponentially.

## 3 – The stability of the non-centered finite-differences schemes

In the previous exercises, we saw that when $a > 0$, a _backward-oriented_ finite-difference scheme yields stability. However, a crucial component in the problem was to give a specific value for $\Delta t$, namely  $t = 0.98 x/|a|$. Would it have been wise to choose a larger or smaller $\Delta t$? Let us check that $\Delta t$ cannot be chosen arbitrarily large: run the program, but now writing

1. $\Delta t = 0.5 \frac{\Delta x}{a}$;
2. $\Delta t = 0.99 \frac{\Delta x}{a}$;
3. $\Delta t = 1.01 \frac{\Delta x}{a}$;
4. $\Delta t = 2 \frac{\Delta x}{a}$;

and check if those values maintain the good stability properties of the code. For example, does there seem to be a threshold in $\Delta t$ for the instability? Note that you need to define `cfl_cut` to 0.5, 0.99, 1.01, and 2.0.

In [ ]:
if __name__ == "__main__":
    cfl_cuts = [0.5, 0.99,1.0, 1.01, 2.0]
    nint = 64
    a = 1
    ddx = nm.deriv_dnw if a<0 else nm.deriv_upw
    bnd_limits = [0,1] if a<0 else [1,0]
    for cfl_cut in cfl_cuts:
        nt = int(52/(cfl_cut*5.2/nint)) + 1 # Number of time steps for the propagating wave to travel through the domain 10 times
        analyse_instability(nint,nt,a=a,ddx=ddx,bnd_limits=bnd_limits, cfl_cut=cfl_cut, animate=False)

In [ ]:
if __name__ == "__main__":
    cfl_cut = 1
    nint = 64
    a = 1
    ddx = nm.deriv_dnw if a<0 else nm.deriv_upw
    bnd_limits = [0,1] if a<0 else [1,0]
    nt = int(52/(cfl_cut*5.2/nint)) + 1 # Number of time steps for the propagating wave to travel through the domain 10 times
    analyse_instability(nint,nt,a=a,ddx=ddx,bnd_limits=bnd_limits, cfl_cut=cfl_cut, animate=False)

<span style="color:Orange">

## Comments

The cfl cut controls whether the Courant-Friedrichs-Lewy (CFL) condition is met. By performing the Von Neumann instability analysis on these FTFS or FTBS schemes we require a $dt < \left|\frac{\Delta x}{a}\right|$. The `cfl_cut` parameter enters as a prefactor in this expression so that we are not right on the onset of instability: `dt = cfl_cut * abs(dx / a)`. With `cfl_cut = 1` we are right on this onset, so we should choose `clf_cut < 1` to be safe from round off errors accumulating over many time steps.

## 4- Optional (A). 

Consider now the following Burgers’ equation, i.e.,

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0   \tag{3}$$

can be seen as yet another case of the equation solved in the previous exercises of this
series by writing:

$$a(x,t,u) = u  \tag{4}$$

The importance of this equation in physics derives in part from the fact that it describes
the motion of a non-accelerated fluid with an arbitrary velocity field at time $t = 0$ and
because it contains the possibility of spontaneous formation of discontinuities. (\*)

We can simply solve equation (1) by modifying the program developed for the previous exercises but now writing $a(x_i, t^n) = u^n_i$. Carry out that modification, and run the program in the domain $x \in (x_0, x_f)$ with $x_0 = −1.4$, $x_f = 2.0$ with the initial condition:

$$u(x,t=0) = A\left[\tanh\left(\frac{x+x_c}{W}\right)-\tanh\left(\frac{x-x_c}{W}\right) \right] + B \tag{5}$$

whereby $A = 1.0$, $x_c = 0.70$, $W = 0.1$, $B = 0.3$. Let the solution evolve until time $t_f = 100$. Explain in physical (or mathematical) terms the solution you get. Change to $A = −0.02$ and explain the result. For this exercise, fill in `nm_lib` functions `evolv_uadv_burgers` and `step_uadv_burgers`. 

In [ ]:
def get_uadv_t0_B(xx: np.ndarray, A: float = 1.0, W: float = 0.1, B: float = 0.3, xc: float = 0.7) -> np.ndarray:
    tanh_term = np.tanh((xx+xc)/W) - np.tanh((xx-xc)/W)
    return A*tanh_term + B

In [ ]:
if __name__ == "__main__":
    x0 = -1.4
    xf = 2.0
    nint = 256
    tf = 100
    xx, _ = utils.get_xx(nint, x0, xf)
    uut0 = get_uadv_t0_B(xx)
    cfl_cut = 0.98
    tt, uunt = nm.evolv_uadv_burgers(xx, uut0, ddx = lambda x,y: nm.deriv_upw(x,y), bnd_limits=[1,0], end_time=tf)
    display(HTML(utils.animate_u(tt[::100], uunt[::100], xx).to_jshtml()))
    plt.close()
    uut0 = get_uadv_t0_B(xx, A=-0.02)
    tt, uunt = nm.evolv_uadv_burgers(xx, uut0, ddx=nm.deriv_upw,bnd_limits=[1,0], end_time=tf)
    display(HTML(utils.animate_u(tt[::100], uunt[::100], xx).to_jshtml()))
    plt.close()

<span style="color:Orange">

### Comments

For the first case with $A=1$, we see how the initial condition evolves in time by traversing in positiv direction (as the advection speed is positiv). The points on either side of the step function moving slower than the points inside the step function. As such the points that initially started with high velocity will eventually reach a boundary between points with high and low velocity and a discontinuity develops. This discontinuity travels as a shock wave, eventually forming a triangular shape with peak at the edge of the shock wave. The peak gradually loses amplitude as the wave travels due to the points behind the peak moving with lower speed than the peak it self.

In the second case with $A=-0.02$, the step function takes on a "negative" shape, and so the sides move with a higher speed than the points inside the step. As such the shock wave develops on the left side of the step, where the sides catches up with the slower moving points inside the step. As the amplitude of the initial step function is lower in this case than the previous, we don't see the same amount of "dampening" of the peak of the shock wave over the same amount of time. I think this is due to the reduced amplitude difference between either side of the discontinuity and that the width of the step is the same in both cases, but the amplitude is much lower in the second case.

### Comments:

In the numerical solution, we see the initial condition patently evolving into something that looks like a discontinuity. We understand that this discontinuity is formed because the characteristic curves $dx_p/dt = a(x, t, u) = u$ are more inclined (i.e., faster) in a spacetime diagram when starting at elements with larger $u$: since the solution is constant along those curves, the faster elements, therefore, catch those in front of them which have lower $u$ (as is the case in the solution you have found numerically here) – so the solution must steepen when that happens.

Mathematically and physically, we should not be too surprised: we know that, for instance, in gas dynamics, weird nonlinear phenomena take place, some of which (the shocks) have to do with the formation of discontinuities or sharp transitions. Numerically, though, we ought to be rather surprised for various reasons:
    
a. the numerical calculation of a very large value of the derivative (as is surely the case across the big jump forming in our calculation) cannot be very accurate. It might even happen, we think, that the program crashed because of _NaNs_ or exceptions, etc, occurring in the calculation. But, in fact, our program happily calculates away ... seemingly forever.

b. when a discontinuity forms, the exact mathematical solution, strictly speaking, cannot be calculated in a simple way anymore: when there is a discontinuity, one must take into account what is called the jump relations, also called internal boundary conditions, across the discontinuity: they give the mutual relation of the variables on either side of the discontinuity. A solution obtained in this way is called a __weak solution__ in mathematics.

c. we finally ask ourselves what the mathematically-exact weak solution would be in this case and whether our simple numerical scheme provides a solution near the exact one despite the obvious difficulty of handling large jumps across the near-discontinuity.

We have to leave these as open questions for discussion at a later point. 

As a final comment, consider the consequences of the fact that the acceleration in gas dynamics contains a term of the general form $u$ $du/dx$ .... Much of the beautiful physics occurring in the universe is due to non-linearities like this one.

## 4- Optional (B) 

For this exercise, do first [ex_4a](https://github.com/AST-Course/AST5110/blob/main/ex_4a.ipynb). 
Consider the same setup as the previous exercise. But now, solve it using the upwind method: 

$$u^{n+1}_j = u^{n}_j - \frac{\Delta t}{\Delta x} u^{n}_j(u^{n}_j- u^{n}_{j+1})$$

And using the Lax method implemented in [ex_4a](https://github.com/AST-Course/AST5110/blob/main/ex_4a.ipynb). What do you see? Argue why you think the solution is not correct. __Hint__ In the first exercise, we disccused about flux conservation.

<span style="color:Orange">

### Comments

Im not sure if I understood this task correctly. I interpret it this way: Using the same setup as in ex_4a, compare the Lax method and the upwind method. This means using the `*_uadv` methods and the initial condition from ex_4a. At this point I must admit I'm a bit confused, as I've been doing both this task and the last task in ex_4a more or less at the same time, so I hope I'll be able to make my self clear in the following:

I've described the evolution of the Lax-method and what we expect previously and in ex_4a. Now compared to the Upwind method we see the two methods differ quite a lot, and I think this is due to the flux conservation as is kindly hinted to in the task. The Upwind method better resolves the very sharp edge formed by the discontinuity, where the Lax-method is more diffusive so the "corners" at the top and bottom of the shock is more curved or smooth. However, the Upwind method is not flux-conservative. As the faster moving points (starting inside the step) catch up to the slower moving points the amplitude of the traveling wave dies out as the shock wave front is not moving. The Lax-method solution on the other hand gets more and more elongated as the shock wave front travels, and the peak of the edge is not as dampened. I think this is because the Lax-method is more flux-conservative. If this was for example a wave in some incompressible fluid, the Upwind method does not conserve the mass or energy of the traversing wave, while the Lax-method seems to (at least) conserve the mass. I've tested this by simply integrating the curve over space for each time step below. As we can see, the area under the curve is constant for the Lax-method, analogous to conserving mass.

I guess this is an example of the types of dilemmas we are faced with when trying to solve different types of physicals problems, there exist no perfect method for all types of problems. While the Upwind method is very stable and non-diffusive, it is not conservative - while the Lax-method is more conservative, albeit highly diffusive.

In [ ]:
if __name__ == "__main__":
    from ex_4a import get_uadv_t0

    x0 = -1.4
    xf = 2.0
    nint = 512
    tf = 100

    xx, dx = utils.get_xx(nint, x0, xf)
    uut0 = get_uadv_t0(xx)
    tt_lax, uunt_lax = nm.evolv_Lax_uadv_burgers(
        xx, uut0, ddx=nm.deriv_cent, bnd_limits=[1, 1], cfl_cut=0.98, end_time=tf
    )
    tt_upw, uunt_upw = nm.evolv_uadv_burgers(xx, uut0, ddx=nm.deriv_upw, bnd_limits=[1, 0], cfl_cut=0.98, end_time=tf)
    sol_dict = {"Lax": (tt_lax, uunt_lax), "Upw": (tt_upw, uunt_upw)}
    display(HTML(utils.animate_us(sol_dict, tt_lax[::10], xx).to_jshtml()))
    plt.close()
    utils.plot_snapshot(sol_dict, xx, np.linspace(tf * 0.1, tf, 3))


In [ ]:
if __name__ == "__main__":
    from scipy.integrate import simpson
    fig, ax = plt.subplots()
    for name, (tt, uu) in sol_dict.items():
        area_under_curve = simpson(uu[:,1:], xx[1:])
        ax.plot(tt, area_under_curve, label=name)
    ax.set_ylabel(r"$\int u(x,t) dx$")
    ax.set_xlabel("time")
    ax.legend()

<span style="color:Orange">

### Disclaimer

In the above plot I integrate over space in each time step, but excluding the first index of the spacial domain. This is due to hwo the periodic boundary conditions are implemented. As we are imposing `uu[0] == uu[nump-2]` and `uu[nump-1] == uu[1]`, we have a duplicate point which sticks out when the shock wave front is right at the boundary between the left and right edge of the plot. This would not have been a problem if I also followed the other point hinted to previously, such that the left hand of the spacial domain `x=x0` coincides with `x[1]`, but I'm not sure how to implement this precisely. I think that with this hint we are including a "ghost" point in the left side of the domain so that the periodic boundaries is better behaved. The way I have implemented it I in a way included the ghost point as the start of my domain. I'll look into this more.

### Comments:
    
See [Stagger mesh](https://github.com/AST-Course/AST5110/wiki/Staggered-mesh) documentation on how a staggered mesh allows keeping the conservation. 